In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import create_ribasim_lumping_network

import pandas as pd
import geopandas as gpd

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", UserWarning)

import networkx as nx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
dhydro_dir = Path(base_dir, "d-hydro")
results_dir = Path(base_dir, "results")
network_name = "tki_zutphen_berkel"

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden", f"{network_name}_afw_eenheden.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

In [ ]:
# Create networkanalysis
network = create_ribasim_lumping_network(
    name=network_name, 
    dhydro_dir=dhydro_dir,
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)
# network.export_to_geopackage()

In [ ]:
# Add simulation sets for winter, summer, etc.
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=dhydro_dir,
    simulations_names=["tki_zuthpen_berkel_basis.dsproj_data"],
    simulation_output_dir="FlowFM\\output",
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="6D"),
);

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data()
# network.export_to_geopackage()

In [ ]:
network.pumps_gdf

MDU file

In [ ]:
import hydrolib.core.dflowfm.mdu.models as hcmdu
workdir = Path(dhydro_dir,'tki_zuthpen_berkel_basis.dsproj_data\FlowFM\input')
mdufile = "FlowFM.mdu"
mdu_path = workdir / mdufile

In [ ]:
## dit werkt nog niet goed; duurt 11 minuten om in te laden en lijkt daarna leeg te zijn??
# model = hcmdu.FMModel(filepath=mdu_path, path_style="windows")

INI file --> structures

In [ ]:
import hydrolib.core.dflowfm.structure.models as hcstruc
workdir = Path(dhydro_dir,'tki_zuthpen_berkel_basis.dsproj_data\FlowFM\input')
inifle = "structures.ini"
inifilepath = workdir / inifle
m = hcstruc.StructureModel(inifilepath)

In [ ]:
df_structures = pd.DataFrame([f.__dict__ for f in m.structure])
df_structures = df_structures.drop(columns=['comments'])
df_structures

lateral files

In [ ]:
import hydrolib.core.dflowfm as hcdfm
workdir = Path(dhydro_dir,'tki_zuthpen_berkel_basis.dsproj_data\FlowFM\input')
lateral_file = "FlowFM_lateral_sources.bc"
lateralfilepath = workdir / lateral_file

forcingmodel_object = hcdfm.ForcingModel(lateralfilepath)
lateral_data = pd.DataFrame([forcing.dict() for forcing in forcingmodel_object.forcing])

In [ ]:
lateral_data

In [ ]:
lateral_data['datablock'][0]

Net files

In [ ]:
# from hydrolib.core.dflowfm.net.models import NetworkModel
# workdir = Path(dhydro_dir,'tki_zuthpen_berkel_basis.dsproj_data\FlowFM\input')
# net_file = "FlowFM_net.nc"
# netfilepath = workdir / net_file

# netwerk = NetworkModel(filepath=netfilepath)

In [ ]:
from hydrolib.core.dflowfm.net.models import Network
workdir = Path(dhydro_dir,'tki_zuthpen_berkel_basis.dsproj_data\FlowFM\input')
net_file = "FlowFM_net.nc"
netfilepath = workdir / net_file
n = Network.from_file(netfilepath)

In [ ]:
n._mesh1d.mesh1d_node_y

In [ ]:
n.__str__

In [ ]:
from shapely.geometry import Point

df1 = pd.DataFrame({'id':n._mesh1d.mesh1d_node_id,'X':n._mesh1d.mesh1d_node_x, 'Y':n._mesh1d.mesh1d_node_y})
df1['coords'] = list(zip(df1['X'], df1['Y']))
df1['coords'] = df1['coords'].apply(Point)
gdf1 = gpd.GeoDataFrame(df1, geometry='coords')
gdf1.plot()

In [ ]:
n._mesh1d.mesh1d_node_y

HIS file --> structures

In [ ]:
df_weircrest = network.his_data.weirgen_crest_level.to_dataframe()
df_weircrest.unstack(level=0).unstack(level=0)

verder met splitnodes en ribasim

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(weirs=True);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    edges=False,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kst_ST80830001', 'kst_ST80810015', # onderdoorlaten bij verdeelpunt De Berkel (Zutphen)
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151', # duikers voor wijk Zuidwijken
        'kdu_DR80950103',
        'kdu_DR80740070', # inlaat twentekanaal
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
        'kst_ST80830045', # weir tussen uitstroompunten Zutphen
    ], 
    edge_ids_to_include=[],
    node_ids_to_include=[
        1784, 2542 # splitsing afleidingskanaal
    ],
    node_ids_to_exclude=[],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    'weir': 'TabulatedRatingCurve', 
    'uniweir': 'TabulatedRatingCurve' ,
    'pump': 'Pump', 
    'culvert': 'TabulatedRatingCurve', 
    'manual': 'ManningResistance',
    'orifice' : 'TabulatedRatingCurve',
    'edge': 'ManningResistance',
}
# specify translation for specific split_nodes to ribasim-nodes
split_node_id_conversion = {
    'sto_AE80770024': 'ManningResistance', 
    'kdu_DR80740070': 'ManningResistance',
    # duikers voor wijk Zuidwijken
    'kdu_DR80940046': 'ManningResistance', 
    'kdu_DR80950043': 'ManningResistance', 
    'kdu_DR80950151': 'ManningResistance', 
}

In [ ]:
# Generate Ribasim model en exporteer naar geopackage
ribasim_model = network.generate_ribasim_model(
    split_node_type_conversion=split_node_type_conversion, 
    split_node_id_conversion=split_node_id_conversion
)
ribasim_model.write(f"{results_dir}/{network.name}")